

   <b> "Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why)".</b>

    – Tim Peters


<img src=../../resources/metaclass.png>

 ###<em>Everything</em> in python is an object: it turns out that this is true of <em>classes themselves</em>.

In [3]:
class DoNothing(object):
    pass

In [4]:
d = DoNothing()
type(d)

__main__.DoNothing

In [5]:
L = [1, 2, 3]
type(L)

list

In [6]:
type(DoNothing)

type

In [7]:
type(tuple), type(list), type(int), type(float)

(type, type, type, type)

In [8]:
type(type)

type

###<em>classes are objects</em>, and they are objects of type <em>```type```</em>
####<code>type</code> is a <em>metaclass</em>: a class which instantiates classes.

.

.

.

.

.

.

.

.

.

###Metaprogramming: Creating Classes on the Fly

What does a typical function do?

 Take some arguments, do some operations, and create & return an object. 
 
 
Then it should be possible to create an object of type <em>type</em> (that is, a class), and returning that


 

.

.

.

.

.

.

In [9]:
def class_factory():
    class Foo(object):
        pass
    return Foo

F = class_factory()
f = F()
print(type(f))

<class '__main__.Foo'>


We can accomplish this by instantiating Foo from type directly.

help(type)

Help on class type in module __builtin__:

class type(object)
 -  type(object) -> the object's type
 
 -  type(name, bases, dict) -> a new type

In [ ]:
class MyClass(object):
    pass

In [ ]:
MyClass = type('MyClass', (), {})

type(name, bases, dct)

<ul>
<li><code>name</code> is a string giving the name of the class to be constructed</li>
<li><code>bases</code> is a tuple giving the parent classes of the class to be constructed</li>
<li><code>dct</code> is a dictionary of the attributes and methods of the class to be constructed</li>
</ul>

In [10]:


class Foo(object):
    i = 4

class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print(b.get_i())



4


In [11]:


Foo = type('Foo', (), dict(i=4))

Bar = type('Bar', (Foo,), dict(get_i = lambda self: self.i))

b = Bar()
print(b.get_i())



4


.

.

.

.

.

.

.

.

.



####Just as we can inherit from and extend a class we've created, we can also:
        
        - inherit from and extend the type metaclass
        - create custom behavior in our metaclass 
        - fry our brains..

.

.

.

.

.

.

In [12]:
class InterfaceMeta(type):
    def __new__(cls, name, parents, dct):
        # create a class_id if it's not specified
        if 'class_id' not in dct:
            dct['class_id'] = name.lower()
        
        # open the specified file for writing
        if 'file' in dct:
            filename = dct['file']
            dct['file'] = open(filename, 'w')
        
        # we need to call type.__new__ to complete the initialization
        return super(InterfaceMeta, cls).__new__(cls, name, parents, dct)

In [13]:
Interface = InterfaceMeta('Interface', (), dict(file='tmp.txt'))

print(Interface.class_id)
print(Interface.file)

interface
<open file 'tmp.txt', mode 'w' at 0x7f384c7da930>


In [14]:
class Interface(object):
    __metaclass__ = InterfaceMeta
    file = 'tmp1.txt'
    
print(Interface.class_id)
print(Interface.file)



interface
<open file 'tmp1.txt', mode 'w' at 0x7f384c7dac00>


In [15]:
type(Interface)

__main__.InterfaceMeta

####Another example: Registering Subclasses

In [16]:


class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls
            
        super(DBInterfaceMeta, cls).__init__(name, bases, dct)



In [17]:
class DBInterface(object):
    __metaclass__ = DBInterfaceMeta
    
print(DBInterface.registry)

{}


In [18]:
class FirstInterface(DBInterface):
    pass

class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print(DBInterface.registry)

{'firstinterface': <class '__main__.FirstInterface'>, 'secondinterface': <class '__main__.SecondInterface'>, 'secondinterfacemodified': <class '__main__.SecondInterfaceModified'>}


e.g.: https://github.com/django/django/blob/master/django/db/models/base.py